Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [X] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [X] Engineer at least two new features. (See below for explanation & ideas.)
- [X] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [2]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module2')

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 156, done.
remote: Total 156 (delta 0), reused 0 (delta 0), pack-reused 156
Receiving objects: 100% (156/156), 19.30 MiB | 13.67 MiB/s, done.
Resolving deltas: 100% (71/71), done.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
     |████████████████████████████████| 92kB 9.9MB/s 
     |████████████████████████████████| 112kB 33.4MB/s 
     |████████████████████████████████| 133kB 45.5MB/s 
     |████████████████████████████████| 57.7MB 368kB/s 
     |████████████████████████████████| 235kB 54.0MB/s 
     |████████████████████████████████| 614kB 46.8MB/s 
     |████████████████████████████████| 235kB 55.8MB/s 
     |████████████████████████████████| 307kB 60.2MB/s 
     |████████████████████████████████| 51kB 23.5MB/s 
     |████████████████████████████████| 215kB 43.6MB/s 
     |█████████

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [40]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

df["created_dt"] = [datetime.strptime(i, "%Y-%m-%d %H:%M:%S") for i in df["created"]]
df.dtypes


bathrooms                      float64
bedrooms                         int64
created                         object
description                     object
display_address                 object
latitude                       float64
longitude                      float64
price                            int64
street_address                  object
interest_level                  object
elevator                         int64
cats_allowed                     int64
hardwood_floors                  int64
dogs_allowed                     int64
doorman                          int64
dishwasher                       int64
no_fee                           int64
laundry_in_building              int64
fitness_center                   int64
pre-war                          int64
laundry_in_unit                  int64
roof_deck                        int64
outdoor_space                    int64
dining_room                      int64
high_speed_internet              int64
balcony                  

In [69]:
import itertools
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.express as px
import plotly.graph_objs as go
from datetime import datetime

def regression_3d(df, x, y, z, iterations=100):
    df_train = df[(df["created_dt"] >= datetime(2016, 4, 21)) & (df["created_dt"] < datetime(2016, 6, 1))] 
    df_test = df[(df["created_dt"] >= datetime(2016, 6, 1)) & (df["created_dt"] < datetime(2016, 7, 1))] 
    
    figure = pltx.scatter_3d(df, x, y, z)

    model = LinearRegression()
    model.fit(df_train[[x, y]], df_train[z])  
    
    x_min, x_max = df_test[x].min(), df_test[x].max()
    y_min, y_max = df_test[y].min(), df_test[y].max()
    x_coords = np.linspace(x_min, x_max, iterations)
    y_coords = np.linspace(y_min, y_max, iterations)
    coords = list(itertools.product(x_coords, y_coords))
    
    predictions = model.predict(coords)
    Z = predictions.reshape(iterations, iterations).T
    figure.add_trace(grpho.Surface(x=x_coords, y=y_coords, z=Z))
    
    b = model.intercept_
    m_1, m_2 = model.coef_
     
    figure.update_layout(title=go.layout.Title(text="Z = {}x + {}y + {}".format(m_1, m_2, b)))
    
    z_pred = df_test[x]*m_1 + df_test[y]*m_2 + b
    z_test = df_test[z]
    RMSE = np.sqrt(mean_squared_error(z_test, z_pred))
    MAE = mean_absolute_error(z_test, z_pred)
    R_2 = r2_score(z_test, z_pred)
    print("Root Mean Squared Error:", RMSE)
    print("Mean Absolute Error:", MAE)
    print("R^2:", R_2)
    
    return figure

regression_3d(df, "bathrooms" , "bedrooms", "price")
 

Root Mean Squared Error: 1219.3390276314135
Mean Absolute Error: 827.5851223659179
R^2: 0.5216288640996356
